### 0️⃣ Initialize notebook variables

Configure the deployment parameters according to your preferences.

In [ ]:
import os
import sys
import json

# Add shared utilities if available
if os.path.exists('shared'):
    sys.path.insert(1, 'shared')

deployment_name = "10x-csa-toolkit"
resource_group_name = f"lab-{deployment_name}"
resource_group_location = "uksouth"  # Change to your preferred location

# Container images configuration
build_number = 0

# MCP Servers configuration - dynamic list of repositories
mcp_servers = [
    {
        "name": "avm-modules",
        "display_name": "AVM MCP Server",
        "repo_url": "https://github.com/nourshaker-msft/avm-mcp-server.git",
        "image_name": "mcp-avm-modules"
    },
    {
        "name": "azure-pricing",
        "display_name": "Azure Pricing MCP Server",
        "repo_url": "https://github.com/nourshaker-msft/mcp-azure-pricing.git",
        "image_name": "mcp-azure-pricing"
    }
]

print('✅ Notebook initialized')
print(f'📦 Deployment name: {deployment_name}')
print(f'📍 Resource group: {resource_group_name}')
print(f'🌍 Location: {resource_group_location}')
print(f'🚀 MCP Servers to deploy: {len(mcp_servers)}')
for server in mcp_servers:
    print(f'   • {server["display_name"]} ({server["name"]})')


### 1️⃣ Verify Azure CLI and connected subscription

Ensure you have the latest version of Azure CLI and are connected to your Azure subscription.

In [ ]:
import subprocess
import json

def run_command(command, description):
    """Execute a shell command and return the result."""
    print(f"🔄 {description}...")
    try:
        result = subprocess.run(
            command,
            shell=True,
            capture_output=True,
            text=True,
            check=True
        )
        return result.stdout
    except subprocess.CalledProcessError as e:
        print(f"❌ Error: {e.stderr}")
        raise

# Get current Azure account
output = run_command("az account show", "Retrieving Azure account information")
account_info = json.loads(output)

current_user = account_info['user']['name']
tenant_id = account_info['tenantId']
subscription_id = account_info['id']

print(f"✅ Connected to Azure")
print(f"👤 Current user: {current_user}")
print(f"🏢 Tenant ID: {tenant_id}")
print(f"📋 Subscription ID: {subscription_id}")

In [ ]:
# Clone all MCP server repositories dynamically
print("🔄 Cloning MCP server repositories...")
for server in mcp_servers:
    try:
        run_command(
            f"git clone {server['repo_url']} ./{server['image_name']}", 
            f"Cloning {server['display_name']} repository"
        )
        print(f"✅ {server['display_name']} cloned to {server['image_name']}")
    except Exception as e:
        print(f"ℹ️ Repository {server['image_name']} may already exist or clone failed: {str(e)}")

print("✅ All repositories processed")


### 2️⃣ Create resource group

Create the Azure resource group where all resources will be deployed.

In [ ]:
# Check if resource group exists
check_rg = f"az group exists --name {resource_group_name}"
rg_exists = run_command(check_rg, "Checking if resource group exists")

if rg_exists.strip() == "true":
    print(f"ℹ️ Resource group '{resource_group_name}' already exists")
else:
    create_rg = f"az group create --name {resource_group_name} --location {resource_group_location}"
    run_command(create_rg, f"Creating resource group '{resource_group_name}'")
    print(f"✅ Resource group '{resource_group_name}' created successfully")

### 3️⃣ Deploy infrastructure using Bicep

Deploy the Azure Container Registry, Container Apps Environment, and Container Apps using the main.bicep template.

In [ ]:
# Create Bicep parameters file with dynamic server configuration
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "location": {"value": resource_group_location},
        "mcpServers": {
            "value": [
                {
                    "name": server["name"],
                    "displayName": server["display_name"],
                    "imageName": server["image_name"]
                }
                for server in mcp_servers
            ]
        }
    }
}

# Write parameters to file
with open('params.json', 'w') as f:
    json.dump(bicep_parameters, f, indent=2)

print("✅ Created parameters file with dynamic server configuration")
print(f"📋 Servers included: {len(mcp_servers)}")
for server in mcp_servers:
    print(f"   • {server['display_name']}")

# Deploy Bicep template
deploy_cmd = f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json"
run_command(deploy_cmd, f"Deploying infrastructure (this may take several minutes)")

print(f"✅ Deployment '{deployment_name}' completed successfully")


### 4️⃣ Get deployment outputs

Retrieve the outputs from the Bicep deployment to use in subsequent steps.

In [ ]:
# Get deployment outputs
get_outputs_cmd = f"az deployment group show --name {deployment_name} --resource-group {resource_group_name} --query properties.outputs"
outputs_json = run_command(get_outputs_cmd, "Retrieving deployment outputs")
outputs = json.loads(outputs_json)

# Extract common values
container_registry_name = outputs['containerRegistryName']['value']
container_registry_login_server = outputs['containerRegistryLoginServer']['value']

# Extract dynamic MCP servers array
mcp_servers_output = outputs['mcpServers']['value']

# Create a lookup dictionary for easy access by server name
servers_lookup = {server['name']: server for server in mcp_servers_output}

# Display deployment outputs
print("✅ Retrieved deployment outputs:")
print(f"🐳 Container Registry: {container_registry_name}")
print(f"📦 MCP Servers deployed: {len(mcp_servers_output)}")

for server_output in mcp_servers_output:
    print(f"\n   {server_output['displayName']}:")
    print(f"   📦 Container App: {server_output['containerAppName']}")
    print(f"   🔗 Server URL: {server_output['url']}/mcp")


In [ ]:
# Install the containerapp extension
try:
    run_command("az extension add --name containerapp --upgrade", "Installing/upgrading containerapp extension")
    print("✅ Container Apps extension installed successfully")
except Exception as e:
    print(f"⚠️ Note: Extension may already be installed or installation failed: {str(e)}")
    print("Continuing with deployment...")

### 5️⃣ Build and deploy MCP server container images

Build the Docker images and push them to Azure Container Registry, then update the Container Apps.

In [ ]:
import time

build_number = build_number + 1

def build_and_deploy_mcp_server(server_config, containerapp_name):
    """Build and deploy an MCP server to Azure Container App."""
    print("=" * 60)
    print(f"🏗️ Building {server_config['display_name']}")
    print("=" * 60)
    
    image_tag = f"{server_config['image_name']}:v{build_number}"
    build_cmd = f"az acr build --image {image_tag} --resource-group {resource_group_name} --registry {container_registry_name} --file {server_config['image_name']}/Dockerfile {server_config['image_name']}/. --no-logs"
    run_command(build_cmd, f"Building and pushing {image_tag}")
    
    print(f"✅ {server_config['display_name']} image built: {image_tag}")
    
    # Update Container App
    update_cmd = f'az containerapp update --name {containerapp_name} --resource-group {resource_group_name} --image "{container_registry_login_server}/{image_tag}"'
    run_command(update_cmd, f"Updating {server_config['display_name']} Container App")
    
    print(f"✅ {server_config['display_name']} deployed successfully")
    print()

# Build and deploy all MCP servers dynamically
for server in mcp_servers:
    # Get the container app name from the servers lookup by matching server name
    server_output = servers_lookup[server['name']]
    containerapp_name = server_output['containerAppName']
    build_and_deploy_mcp_server(server, containerapp_name)

print("=" * 60)
print(f"🎉 All {len(mcp_servers)} MCP servers deployed successfully!")
print("=" * 60)


### 6️⃣ List available MCP tools

Connect to the MCP servers using the MCP protocol and list available tools.

In [ ]:
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

import asyncio
import nest_asyncio
nest_asyncio.apply()

async def list_mcp_tools(server_url, server_name):
    """Connect to an MCP server and list available tools."""
    print(f"🔌 Connecting to {server_name}...")
    print(f"🌐 Server URL: {server_url}/mcp")
    try:
        async with streamablehttp_client(f"{server_url}/mcp") as streams:
            async with ClientSession(streams[0], streams[1]) as session:
                await session.initialize()
                response = await session.list_tools()
                tools = response.tools
                
        print(f"✅ Connected to {server_name}")
        print(f"📋 Available tools ({len(tools)}):")
        for tool in tools:
            print(f"  • {tool.name}")
            if hasattr(tool, 'description') and tool.description:
                print(f"    {tool.description[:100]}...")
        print()
        return tools
    except Exception as e:
        print(f"❌ Error connecting to {server_name}: {str(e)}")
        print()
        return []

print("=" * 60)
print("🔧 Discovering MCP Tools")
print("=" * 60)
print()

# List tools from all servers dynamically
all_tools = {}
for server_output in mcp_servers_output:
    tools = asyncio.run(list_mcp_tools(server_output['url'], server_output['displayName']))
    all_tools[server_output['name']] = tools

# Calculate total tools
total_tools = sum(len(tools) for tools in all_tools.values())

print("=" * 60)
print(f"✅ Total tools available: {total_tools}")
for server_output in mcp_servers_output:
    print(f"   • {server_output['displayName']}: {len(all_tools[server_output['name']])} tools")
print("=" * 60)


In [ ]:
# Generate MCP configuration file for VS Code dynamically
mcp_config = {
    "servers": {}
}

# Add all servers dynamically
for server_output in mcp_servers_output:
    mcp_config["servers"][server_output['name']] = {
        "type": "http",
        "url": f"{server_output['url']}/mcp"
    }

# Write to mcp.json file
with open('../.vscode/mcp.json', 'w') as f:
    json.dump(mcp_config, f, indent=2)

print("✅ MCP configuration file created: .vscode/mcp.json")
print()
print("📋 MCP Server Configuration:")
print(json.dumps(mcp_config, indent=2))
print()
print("📝 To use with VS Code MCP Extension:")
print("   Copy the contents of mcp.json to:")
print("   ~/.config/Code/User/globalStorage/saoudrizwan.claude-dev/settings/cline_mcp_settings.json")
print()
print("🔗 Server URLs:")
for server_output in mcp_servers_output:
    print(f"   • {server_output['displayName']}: {server_output['url']}/mcp")


### 7️⃣ Usage Examples

#### Using with VS Code MCP Extension

Add these MCP servers to your VS Code configuration (`~/.config/Code/User/globalStorage/saoudrizwan.claude-dev/settings/cline_mcp_settings.json`):

```json
{
  "mcpServers": {
    "avm-modules": {
      "type": "http",
      "url": "YOUR_AVM_SERVER_URL/mcp"
    },
    "azure-pricing": {
      "type": "http",
      "url": "YOUR_PRICING_SERVER_URL/mcp"
    }
  }
}
```

#### Using with Claude Desktop

Add to your Claude Desktop configuration:

```json
{
  "mcpServers": {
    "avm-modules": {
      "command": "http",
      "args": ["YOUR_AVM_SERVER_URL/mcp"]
    },
    "azure-pricing": {
      "command": "http",
      "args": ["YOUR_PRICING_SERVER_URL/mcp"]
    }
  }
}
```

#### Example Queries

Once connected, you can ask:

**For AVM Modules:**
- "List all available Azure Verified Modules"
- "Show me details about the storage account AVM module"

**For Azure Pricing:**
- "What are the available Azure service families?"
- "How much does a Standard D2s v3 VM cost per month in East US?"
- "Compare pricing for Azure App Services in different regions"

### 8️⃣ Clean up resources

When you're finished, remove all deployed resources to avoid unnecessary charges.

In [ ]:
# Uncomment the following lines to delete the resource group and all resources
# WARNING: This will permanently delete all resources in the resource group!

# delete_rg_cmd = f"az group delete --name {resource_group_name} --yes --no-wait"
# run_command(delete_rg_cmd, f"Deleting resource group '{resource_group_name}'")
# print(f"🗑️ Resource group '{resource_group_name}' deletion initiated")
# print("Note: Deletion may take several minutes to complete")

print("ℹ️ To delete all resources, uncomment the code above and run this cell")

---

## Summary

You've successfully deployed two MCP servers to Azure Container Apps:

1. **AVM MCP Server** - Provides tools to discover and query Azure Verified Modules
2. **Azure Pricing MCP Server** - Provides tools to query Azure pricing information

Both servers are now accessible via HTTPS and can be integrated with:
- VS Code Copilot with MCP extensions
- Claude Desktop
- Any MCP-compatible client
- Semantic Kernel applications

The deployment includes:
- ✅ Azure Container Registry for image storage
- ✅ Container Apps Environment with logging
- ✅ Two Container Apps running the MCP servers
- ✅ Managed Identity for secure operations
- ✅ HTTPS ingress for external access

**Next Steps:**
- Configure your MCP client with the server URLs
- Test the tools with your AI assistant
- Monitor logs in Log Analytics workspace
- Scale the Container Apps based on usage